In [ ]:
import siibra
from nibabel import Nifti1Image
from nilearn import plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## Create an artificial input image volume with activation blobs

We sample anatomical locations from known regions in the DiFuMo64 parcellation, which represents functional modules, and create a Nifti volume with Gaussian activation blobs at these locations as an artificial input signal.

In [ ]:
space = siibra.spaces.MNI152_2009C_NONL_ASYM
difumo64 = siibra.parcellations.DIFUMO_64.get_map(space)
regions = ['Heschl', 'Fusiform']

In [ ]:
# create empty volume in the reference space
tpl = space.get_template().fetch()
blobimg = Nifti1Image(np.zeros_like(tpl.get_fdata()), tpl.affine)
blobarr = np.asanyarray(blobimg.dataobj)

gaussian_blob = siibra.volumes.util.create_gaussian_kernel(4)
w = gaussian_blob.shape[0]//2

for regionspec in regions:
    
    # get voxel position of a point in the region
    point = difumo64.decode_region(regionspec).centroids(space)[0]
    x, y, z = map(int, point.transform(np.linalg.inv(tpl.affine)))

    # place gaussian blob at that position
    blobarr[x-w:x+w+1, y-w:y+w+1, z-w:z+w+1] = gaussian_blob
    
# plot the resulting blob image
plotting.plot_glass_brain(blobimg)

## Run anatomical assignment


The assignment quereis datasets from the EBRAINS Knowledge Graph, so we have to pass a valid access token. This requires you to register an EBRAINS account, for more details see the documentation at https://siibra-python.readthedocs.io/en/latest/start.html#access-to-ebrains.

In [ ]:
siibra.fetch_ebrains_token()

In [ ]:
from siibra_toolbox_neuroimaging import AnatomicalAssignment
assignment = AnatomicalAssignment()

In [ ]:
assignments, component_mask = assignment.analyze(blobimg)
assignment.create_report(assignments, blobimg, component_mask)